In [1]:
#from sentence_transformers import CrossEncoder
#from pytorch_modelsize import SizeEstimator
import pandas as pd
import numpy as np
from tqdm import tqdm
import heapq
import os

tqdm.pandas()
#model = CrossEncoder('cross-encoder/stsb-roberta-large')
import codecs
import hickle as hkl
import pickle

print(os.getcwd())

C:\Users\ariel\Documents\PHD\GitHub\semantic_similarity\notebooks


In [59]:
def save_sentences(data, filename="sentences.pkl",path = r"C:\Users\ariel\Documents\PHD\GitHub\semantic_similarity\notebooks"): #saves data with a given filename in a path
    if not os.path.exists(os.path.join(path,filename)):
        print(data)
        pickle.dump(data, open(os.path.join(path,filename), 'wb'))
    else:
        print("the file already exists")

def read_conversations(names=[]): #It reads a text file that contains subtitles referred to a conversation between 2 or more people. It expects that each time a person talks, we have
                                  #one or more lines with the format name: sentence
    #the person pronounced, while idx is an incremental number that is incremented each time a person talks
    path = r"C:\Users\ariel\Documents\PHD\GitHub\semantic_similarity\notebooks\[English] 1.5 HOUR English Conversation Lesson [DownSub.com].txt"
    names = ['Vanessa', 'Dan']
    sentences = dict((name,[]) for name in names)
    #this dictionary has names of actors as keys, as value they have a tuple (sentence, idx) where sentence is the sentence that

    with open(path, "r", encoding='utf-8') as f:
        subs = [line.rstrip() for line in f if line.rstrip() != '']  #removes \n and "\" before apostroph
        i = 0
        turn = 1 # counter that counts the turns ( turn = turn + 1 each time a person finish to talk and the other starts)
        while i<len(subs):
            actual_name_list = [name for name in names if name+":" in subs[i]] #is the person that is currently talking
            if len(actual_name_list) == 1: # A person is talking. if the person is the same in the next rows, the sentence is composed by all the the words in these rows put together
                name = actual_name_list[0]
                sentence = subs[i][len(actual_name_list[0])+1:]    #remove "Name:"
                #print(i,len(subs)-1)
                if i < len(subs)-1: #avoid exceeding the list of subtitles
                    i = i + 1
                    next_name_list = [name for name in names if name+":" in subs[i]]
                    while len(next_name_list) == 0 and i != len(subs)-1 or (len(next_name_list)==1 and next_name_list[0] == name):  #same person is talking
                        sentence = sentence + ' ' + subs[i]
                        i = i + 1
                        next_name_list = [name for name in names if name+":" in subs[i]]
                        #print( len(next_name_list), next_name_list, name, i != len(subs), i , len(subs)-1)
                    #print("OUT!")

                sentences[name].append((sentence,turn))
                turn = turn + 1
                #print(i,len(subs)-1)
            else: # If we have for any reason more names or no names at the start of the sentence, we skip it
                i = i+1
    return sentences

all_sentences = read_conversations()
path = r"C:\Users\ariel\Documents\PHD\GitHub\semantic_similarity\notebooks"
save_sentences(all_sentences,path=path)
filename = "sentences.pkl"
data = pickle.load(open(os.path.join(path,filename),'rb'))
print(data)

the file already exists
{'Vanessa': [(" Hi, I'm Vanessa from SpeakEnglishWithVanessa.com.", 1), (" My husband, who is also going to be giving another perspective in today's conversation. Today we're bringing you an amazing, long, English conversation, so prepare your ears, prepare your mind. We're going to be talking about 12 different topics, and hopefully providing some new expressions and new ways to think about life. I don't know about that, but at least some English help for you.", 3), (' Yes. All right, are you ready to get started?', 5), (" Let's go. Our first topic is family, and my question is, who do you think that you're the most like?", 7), (' Yes.', 9), (' Yeah, both.', 11), (' Okay.', 13), (' Yeah.', 15), (" Yeah, I'd say you have shades of your dad.", 17), (' Yeah. Yeah, I think you have shades of your dad. Yeah.', 19), (" Ah, he's got a silly side though.", 21), (' But your dad is even more stress free.', 23), (' And your mom, and brother, and sister were saying, "Come 

In [60]:
class Subs_classification:

    # filename contains the subtitles filepath
    # refs contains the dictionary with all the reference sentences
    # model variable contains the model that will be used for the semantic similarity
    # winsize represents the number of words inside the sentence that will be used as comparison
        # with the reference sentences. Not necessary in the "fullstop_classification" mode
    # stride indicates the amount of words skipped to traslate the window. Not necessary in "fullstop_regression" mode
    # thresold represents the value over which we consider the prediction of the model "valid", i.e. when
        # reference sentence and sentence are similar enough
    # mode is a string that represents the modality that we want to use as comparison:
        # "fullstop_regression" compares a reference sentence with a sentence between two full stop
        # "window_regression" compares a reference sentence with a sentence made up by the words inside the window
        # "mixed_rgression" compares a reference sentence with a sentence made up by the words inside the window but
            # it also find relations between windows inside a sentence between two full stops. The last window does not take
            # the words after the full stop

    def __init__(self, filename = '', refs = {},
                 model = CrossEncoder('cross-encoder/stsb-roberta-base'),
                 winsize = 5, stride = 2, threshold = 0.3, mode = "fullstop_regression"):

        self.filepath = filename
        self.subs = ''           #self.read_file(filename)
        self.refs = refs
        self.winsize = winsize
        self.stride = stride
        self.threshold = threshold
        self.mode = mode

    # used to read the "filename" file. It reads a row at time and then puts all the rows together
    def read_file(self, filename):

        with open(subtitles_path,"r",encoding='utf-8') as f:
            subs = [line.rstrip() for line in f if line.rstrip()!='']  #removes \n
            subs_final = ''
            #subs_final = [subs_final + s + ' ' for s in subs]
            for el in subs:
                subs_final = subs_final + el + ' '  #puts all the subs together
            self.subs = subs_final
            return self.subs


    def sub_classifier(self, mode='', method='', sentences = '', type = 'conversation'):

        if mode!='':
            self.mode = mode
        if method!='':
            self.method = method

        if self.mode == "fullstop_regression":
            return self._fullstop_regression(self.method)
        elif self.mode == "window_regression":
            return self._window_regression(self.method)
        elif self.mode == "mixed_regression":
            return self._mixed_regression(self.method, sentences, type)
        else:
            print("Choose a right mode and a right method")


    def _append_results(self, thresholds_dict, value, ref, sentence):

        thresholds_dict["value"].append(value)
        thresholds_dict["ref"].append(ref)
        thresholds_dict["sentence"].append(sentence)

        return thresholds_dict


    def _best_threshold(self,local_th, all_th, sentence):

        if len(local_th["value"])!=0: #then we have at least one reference match
            max_threshold = max(local_th["value"])
            index = local_th["value"].index(max_threshold)
            all_th = self._append_results(all_th, max_threshold,
                                            local_th["ref"][index], sentence)
        #else:
            #all_th = self._append_results(all_th, None,  None, sentence)
        local_th = {"value":[], "ref":[], "sentence":[]} #I don't need the local thresholds anymore


        return local_th, all_th

    def _window_regression(self, method = "best_threshold"):

        subs = self.read_file(self.filepath)    #all the subtitles
        splits = subs.split()[0:200]
        #best_th = {"value":[], "ref":[], "sentence":[]}
        all_best_thresholds = {"value":[], "ref":[], "sentence":[]}
        local_thresholds = {"value":[], "ref":[], "sentence":[]}
        all_local_th = {}

        for i in range(int(len(splits)/self.stride)):
            sentence = " ".join(splits[i*self.stride : i*self.stride + self.winsize]) #sentence that will be compared with all the ref sentences
            for key in self.refs.keys():    #each key represents a topic related to a specific gesture, each element in the key is a sentence that can generate that gesture
                for ref_sentence in self.refs[key]:   #for each reference, do a comparison with a sentence inside subtitles. If they are similar (>threshold) then append the result to local_thresholds
                    res = model.predict([sentence,ref_sentence])
                    if res > self.threshold:
                        local_thresholds = self._append_results(local_thresholds, res, ref_sentence, sentence)

                if method == "best_threshold_for_each_topic":     #for each topic, find the reference that is the most similar to the sentence (if there is anyone) and put results in all_best_thresholds
                    local_thresholds, all_best_thresholds = self._best_threshold(local_thresholds, all_best_thresholds, sentence)

            if method == "best_threshold":  #for each window, find the best reference of the best topic that is the most similar to the sentence (if there is anyone
                local_thresholds, all_best_thresholds = self._best_threshold(local_thresholds, all_best_thresholds, sentence)

        if method == "best_threshold" or method == "best_threshold_for_each_topic":
            return all_best_thresholds
        elif method == "all_thresholds":
            return local_thresholds


    def _fullstop_regression(self, method = "best_threshold"):

        subs = self.read_file(self.filepath)
        splits = subs.split(".")[0:200]
        all_best_thresholds = {"value":[], "ref":[], "sentence":[]}
        local_thresholds = {"value":[], "ref":[], "sentence":[]}

        for sentence in splits:
            for key in self.refs.keys():
                for ref_sentence in self.refs[key]:
                    res = model.predict([sentence,ref_sentence])
                    if res > self.threshold:
                        local_thresholds = self._append_results(local_thresholds, res, ref_sentence, sentence)

                if method == "best_threshold_for_each_topic":     #for each topic, find the reference that is the most similar to the sentence (if there is anyone) and put results in all_best_thresholds
                    local_thresholds, all_best_thresholds = self._best_threshold(local_thresholds, all_best_thresholds, sentence)

            if method == "best_threshold":  #for each window, find the best reference of the best topic that is the most similar to the sentence (if there is anyone
                local_thresholds, all_best_thresholds = self._best_threshold(local_thresholds, all_best_thresholds, sentence)

        if method == "best_threshold" or method == "best_threshold_for_each_topic":
            return all_best_thresholds
        elif method == "all_thresholds":
            return local_thresholds


    def _mixed_regression(self, method = "best_threshold", sentences='', type = 'conversation'):
        #TODO save the starting index of the matched words to avoid later correction
        #subs = self.read_file(self.filepath)
        subs = sentences
        all_local_all_sentences=[]  #it contains all the local results for all possible windows for all the sentences
        all_best_all_sentences=[]  #it contains all the local results for all possible windows for all the sentences
        #print(subs)
        if type == 'conversation':      # In this case sentences are of type (speaker,sentence,turn), otherwise we have only a list of sentences
            conv_subs=[]
            for key in list(subs.keys()):
                speaker = (key,)
                tuples = [speaker + element for element in subs[key]]
                conv_subs.append(tuples)
                #subs[key] = [ for (sentence,turn) in subs[key]]
            #subs = conv_subs[0][0:10] + conv_subs[1][0:10] #take 10 sentences for 2 speakers
            subs = conv_subs[1][:100] + conv_subs[0][:100]
            #print(subs)

        #print(subs)
        #subs = subs[0:1] + subs[2:3] #+ subs[8:13]
        #print(subs)
        sentences_processed = 0
        filename_best = "all_processed_sentences_best.pkl"
        filename_local = "all_processed_sentences_local.pkl"
        window_max_len = 10
        path = r"C:\Users\ariel\Documents\PHD\GitHub\semantic_similarity\notebooks"

        print("loading file with local matches...")
        if(os.path.exists(os.path.join(path,filename_best))):    #load data already processed
            all_best_all_sentences = pickle.load(open(os.path.join(path,filename_best),'rb'))
        else:
            pickle.dump(all_best_all_sentences, open(os.path.join(path,filename_best),'wb'))
        print("File with local matches loaded! Loading now the file with best matches...")
        if(os.path.exists(os.path.join(path,filename_local))):    #load data already processed
            all_local_all_sentences = pickle.load(open(os.path.join(path,filename_local),'rb'))
        else:
            pickle.dump(all_local_all_sentences, open(os.path.join(path,filename_local),'wb'))
        print("both files loaded")
        #print(all_best_all_sentences)

        for sentence in subs:
            all_local_th = {}
            all_best_th = {}
            best_th = {"value":[], "ref":[], "sentence":[]}
            local_th = {"value":[], "ref":[], "sentence":[]}    #contains all the possible matches for a given winsize
            all_locals = {"value":[], "ref":[], "sentence":[]}  #contains all the possible matches for a given winsize, but differently from local_th, it
                                                                #is does not become empty if method is "best_threshold" or "best_threshold_for_each_topic"
            window_size = self.winsize
            skip_flag = False
            if type == 'conversation':
                #print(sentence,len(sentence))
                speaker = sentence[0]
                turn = sentence[2]
                sentence = sentence[1]
                for i in range(len(all_best_all_sentences)):
                    if((sentence,speaker,turn) == all_best_all_sentences[i]["whole sentence,speaker,turn"]): #we already have this data
                        skip_flag = True
                        print(f"skipped: {(sentence,speaker,turn)}")
                        sentences_processed = sentences_processed + 1
                if skip_flag == True:
                        continue
                else:
                    all_local_th["whole sentence,speaker,turn"] = (sentence,speaker,turn)
                    all_best_th["whole sentence,speaker,turn"] = (sentence,speaker,turn)
            else:
                for i in range(len(all_best_all_sentences)):
                    if(sentence == all_best_all_sentences[i]["whole sentence"]):
                        skip_flag = True
                        print(f"skipped: {sentence}")
                        sentences_processed = sentences_processed + 1
                        continue
                if skip_flag == True:
                    continue
                else:
                    all_local_th["whole sentence"] = sentence
                    all_best_th["whole sentence"] = sentence

            sentence_splits = sentence.split()
            while window_size <= len(sentence_splits) and window_size <= window_max_len:
                #print(window_size)
                #print(self.winsize)
                #print(f"HERE:{sentence_splits}")
                #print(len(sentence_splits))
                n_windows = int((len(sentence_splits) - window_size + 1) / self.stride)
                #print(n_windows)
                for i in range(n_windows):  #use the window inside the sentence
                    sub_sentence = " ".join(sentence_splits[i*self.stride : i*self.stride + window_size]) #sentence that will be compared with all the ref sentences
                    for key in self.refs.keys():    #for all types of references
                        for ref_sentence in self.refs[key]: #for all references
                            #print(self.winsize, sub_sentence)
                            res = model.predict([sub_sentence,ref_sentence])
                            if res > self.threshold:    #then we have a match
                                local_th = self._append_results(local_th, res, ref_sentence, sub_sentence) #put the result in local_th
                                all_locals = self._append_results(all_locals, res, ref_sentence, sub_sentence) #same operation but this list do not become empty if method is "best_threshold_for_each_topic" or "best_threshold". Used for saving purposes

                        if method == "best_threshold_for_each_topic":     #for each topic, find the reference that is the most similar to the sentence (if there is anyone) and best_th_topic
                            local_th, best_th = self._best_threshold(local_th, best_th, sub_sentence)

                    if method == "best_threshold":  #for each window, find the best reference of the best topic that is the most similar to the sentence (if    there is anyone)
                        local_th, best_th = self._best_threshold(local_th, best_th, sub_sentence)
                        #print(best_th)

                all_local_th[f"window {window_size}"] = all_locals  #save results for each window size
                all_best_th[f"window {window_size}"] = best_th
                local_th = {"value":[], "ref":[], "sentence":[]} #I don't need the local thresholds anymore
                best_th = {"value":[], "ref":[], "sentence":[]} #I don't need the best thresholds anymore
                window_size = window_size + 1
            #print(all_best_th)
            all_local_all_sentences.append(all_local_th)
            all_best_all_sentences.append(all_best_th)
            sentences_processed = sentences_processed + 1
            print("writing to file...")
            pickle.dump(all_best_all_sentences, open(os.path.join(path,filename_best),'wb'))
            pickle.dump(all_local_all_sentences, open(os.path.join(path,filename_local),'wb'))
            print(f"file written {sentence, speaker, turn}, sentence processed: {sentences_processed}")
            #print(f"processed a sentence:{all_best_th}")
            #print(all_local_th)


        if method == "best_threshold" or method == "best_threshold_for_each_topic":
            return all_best_all_sentences
        elif method == "all_thresholds":
            return all_local_all_sentences

NameError: name 'CrossEncoder' is not defined

In [ ]:
th = 0
filename = "sentences.pkl"
path = r"C:\Users\ariel\Documents\PHD\GitHub\semantic_similarity\notebooks"
all_sentences = pickle.load(open(os.path.join(path,filename),'rb'))
classifier = Subs_classification(filename = subtitles_path, refs = all_refs, winsize = 1, stride = 1,
                                  threshold = th, mode = "mixed_regression")
result = classifier.sub_classifier(mode='', method="best_threshold_for_each_topic", sentences = all_sentences , type = 'conversation')
#print(result)

In [2]:
#reference sentences that will be compared by the model with a sentence inside subtitles

reference_sentence_greet = ["hello", "hey there", "I greet you", "Greetings!", "Namaste", "good day", "goodbye", "see you"]
reference_sentence_faster = ["this is boring, go faster", "please talk faster", "continue please", "I have no time, finish the speech please"]
reference_sentence_goaway = ["let me alone", "do not disturb me", "I need some privacy", "go away", "you annoyed me"]
reference_sentence_cigarette = ["do you like to smoke?", "smoking is not healthy", "cigarette", "where are cigarettes",
                                "smoking is the cause of many diseases", "I like to smoke", "smoking"]
reference_sentence_idk = ["I don't know", "I don't understand"]
reference_sentence_no = ["I don't agree", "I don't like it", "no", "I don't want", "it's not ok", "It is not the same for me"]
reference_sentence_yes = ["yes.", "yeah.", "I agree", "It is the same for me", "I like it", "ok", "I want it"]
reference_sentence_deitic = ["see there", "see that", "look at your left", "look at your right", "look over your head", "look behind"]
reference_sentence_stop = ["slow down", "don't go further", "stop", "go slower", "stop talking", "talk slower"]
reference_sentence_adaptor = ["my arm", "my hand", "my head","my knee", "my leg", "my stomach", "my chest",
                             "my feet", "my eyes", "me", "I am"]
reference_sentence_telephone = ["I have to do a call", "I want to call my friend", "telephone number", "call me"]
reference_sentence_come = ["come to me", "come here", "come closer"]
reference_sentence_drink = ["do you want a drink?", "I want to drink", "I want some water", "can you give me a drink?",
                            "let's take a drink", "I am thirsty","let's take a drink"]
reference_sentence_exulting = ["I won!", "I am very happy!"]
reference_sentence_link = ["they are a couple", "they are lovers", "they are similar", "they understand each other"]
reference_sentence_apologize = ["excuse me", "please forgive me", "I am sorry"]
reference_sentence_attention = ["pay attention", "listen me", "look at here"]
reference_sentence_beg = ["please listen me", "I beg you"]
reference_sentence_careless = ["I don't care", "It's not my problem"]
reference_sentence_impossible = ["How is it possible?", "It's impossible"]
reference_sentence_walk= ["I like to walk", "do you like to walk?", "I like trekking", "I want to do some trekking",
                           "I like hiking", "I often go out for a walk"]
reference_sentence_run = ["I like to run","do you like to run?", "where you will run?", "I often run", "I want to run later"]
reference_sentence_thin = ["that person is thin", "I'm thin", "that object is thin"]
reference_sentence_later = ["let's meet later", "see you tomorrow", "next week", "next month", "next year", "do it later"]
reference_sentence_lot = ["it is very", "there is a lot", "it is too much"]
reference_sentence_time = ["what time is it?", "can you tell me the time", "It's late", "you are late"]
reference_sentence_clap = ["you did very well!", "compliments!", "congratulations!"]
reference_sentence_ita = ["What do you want?", "what are you saying?"]


all_refs = {"r_greet":reference_sentence_greet, "r_faster":reference_sentence_faster, "r_goaway":reference_sentence_goaway,
           "r_cigarette":reference_sentence_cigarette, "r_idk":reference_sentence_idk, "r_no":reference_sentence_no,
           "r_yes":reference_sentence_yes, "r_deitic":reference_sentence_deitic,"r_run":reference_sentence_run,
           "r_stop":reference_sentence_stop, "r_adaptor":reference_sentence_adaptor, "r_telephone":reference_sentence_telephone,
           "r_come":reference_sentence_come, "r_drink":reference_sentence_drink, "r_exulting":reference_sentence_exulting,
            "r_link":reference_sentence_link, "r_apologize":reference_sentence_apologize, "r_attention":reference_sentence_attention,
            "r_beg":reference_sentence_beg, "r_careless":reference_sentence_careless, "r_impossible":reference_sentence_impossible,
            "r_walk":reference_sentence_walk,"r_thin":reference_sentence_thin, "r_later":reference_sentence_later,
            "r_lot":reference_sentence_lot, "r_time":reference_sentence_time, "r_clap":reference_sentence_clap,"r_ita":reference_sentence_ita}

In [ ]:
def avg_windows(results, all_refs, new_th=0):

    average_windows = {} #it will contain the average window for each topic, the standard deviation, the number of matches, the average and std of the values of the matches
    all_topics_data = {} #it contains all the window sizes and all the values for each topic so that we can compute sum and std at the end and put the result in the previous dict
    for topic in all_refs: #init the average windows
        average_windows[topic] = {'avg_window':0, 'std_window':0, 'n_matches':0, 'avg_value':0, 'std_value':0}
        all_topics_data[topic] = {'windows':[], 'values':[]}
    for i,res in enumerate(results):+
        sentence = res['whole sentence,speaker,turn'][0]
        #speaker = res['whole sentence,speaker,turn'][1]
        #turn = res['whole sentence,speaker,turn'][2]
        window_dim = int(list(res.keys())[-1][-1]) #the last key of res contains data of the largest window that was used for the sentence. The last value of the key is window dim
        #window_dim = len(sentence_splits)
        #print(window_dim)
        for j in range(1,window_dim+1):
            values = res[f"window {j}"]['value']
            th_indexes = [i for i in range(len(values)) if values[i] > new_th] #returns the indexes of the values that are greater than new_th
            tmp_values = list(map(values.__getitem__,th_indexes)) #take all the values greater than new_th
            tmp_ref = list(map(res[f"window {j}"]['ref'].__getitem__,th_indexes)) #take all the references that returns a value greater than new_th
            #tmp_sentence = list(map(res[f"window {j}"]['sentence'].__getitem__,th_indexes))
            for k,value in enumerate(tmp_values): #for each value that is greater than the threshold
                ref = tmp_ref[k]
                topic = [key for key in all_refs.keys() for t_ref in all_refs[key] if ref == t_ref][0] #extract the topic associated with reference
                average_windows[topic]['n_matches'] = average_windows[topic]['n_matches'] + 1 #increment the number of matches
                all_topics_data[topic]['windows'].append(j)
                all_topics_data[topic]['values'].append(value)

    for topic in all_refs:  #now we can compute all the averages and standard deviations
        if average_windows[topic]['n_matches'] != 0:
            average_windows[topic]['avg_window'] = round(np.mean(np.array(all_topics_data[topic]['windows'])),3)
            average_windows[topic]['std_window'] = round(np.std(np.array(all_topics_data[topic]['windows'])),3)
            average_windows[topic]['avg_value'] = round(np.mean(np.array(all_topics_data[topic]['values'])),3)
            average_windows[topic]['std_value'] = round(np.std(np.array(all_topics_data[topic]['values'])), 3)

    return average_windows



path = r"C:\Users\ariel\Documents\PHD\GitHub\semantic_similarity\notebooks"
filename_best = "all_processed_sentences_best.pkl"
filename_local = "all_processed_sentences_local.pkl"


if(os.path.exists(os.path.join(path,filename_best))):    #load data already processed
    results = pickle.load(open(os.path.join(path,filename_best),'rb'))
else:
    print("Invalid path")

average_windows = avg_windows(results, all_refs, new_th = 0.15)
for i,el in enumerate(list(average_windows.keys())):
    average_windows[el]['priority'] = np.random.randint(1000) #add a priority to the topic (now they are random, the order has yet to be decided)

filename_res = "average_windows_200_sentences.pkl"
print("writing to file... \n \n")
pickle.dump(average_windows , open(os.path.join(path,filename_res),'wb'))

for el in list(average_windows.keys()):
    print(f"{el}: {average_windows[el]}", "\n", "\n")

In [ ]:
for i,el in enumerate(list(res.keys())):
    res[el]['priority'] = np.random.randint(1000) #add a priority to the topic (now they are random, the order has yet to be decided)
topics_ordered = sorted(res.items(), key=lambda x: x[1]['priority'], reverse=False) #order the topics on priority values
#print(topics_ordered[0][1]['avg_window'])

n_possible_topics = 3
index = 5
a = range(0,1000)
b = np.random.randint(1000,size=1000)
values = [i for i in b[index : index + n_possible_topics]]
indexes = [i for i in a[index : index + n_possible_topics]]
#for i,j in zip(values,indexes):
    #print(i,j)


a = [0,2,3,4,5,8]
b = [a[i]]
print(b)

In [ ]:
# give priority to each topic. Use the average window of each topic to take data of the sentence. If, for a topic, we have overlapping windows, chose the one that returns the best value.
# After we choose a topic given the value, the window size and the priority, all the words of that window can't be assigned to any other topic
path = r"C:\Users\ariel\Documents\PHD\GitHub\semantic_similarity\notebooks"
filename_best = "all_processed_sentences_best.pkl"
filename_local = "all_processed_sentences_local.pkl"
filename_windows = "average_windows_200_sentences.pkl"

if(os.path.exists(os.path.join(path,filename_best))):    #load data already processed
    results = pickle.load(open(os.path.join(path,filename_best),'rb'))
else:
    print("Invalid path")

if(os.path.exists(os.path.join(path,filename_windows))):    #load data already processed
    average_windows = pickle.load(open(os.path.join(path,filename_windows),'rb'))
else:
    print("Invalid path")


def process_results_priority_policy_avg_windows(avg_windows = average_windows ,result = results, new_th=0.8):
    final_labels=[]
    for el in result:
        sentence_labels = {'sentence':'', 'topics&ref':[], 'matching_words':[], 'values':[], 'priority_levels':[]} #it contains a sentence labelled
        sentence = el['whole sentence,speaker,turn'][0]
        speaker = el['whole sentence,speaker,turn'][1]
        turn = el['whole sentence,speaker,turn'][2]
        sentence_labels['sentence'] = sentence
        sentence_splits = sentence.split()
        final_word_index = len(sentence_splits)
        current_word_index = 0
        break_flag = False #if true,  we found the right label to a set of words in the sentence, so we can continue
        topics_ordered = sorted(avg_windows.items(), key=lambda x: x[1]['priority'], reverse=True) #order the topics on priority values
        topic_index = 0
        while topic_index < len(topics_ordered) and current_word_index <= final_word_index:
            topic_name = topics_ordered[topic_index][0]
            window_dim = round(topics_ordered[topic_index][1]['avg_window'])
            if current_word_index + window_dim > final_word_index: #we can't try with this topic since we reached the end of the sentence
                topic_index = topic_index + 1
                continue
            words_in_window = ' '.join(sentence_splits[current_word_index : current_word_index + window_dim])
            current_values = el[f"window {window_dim}"]['value']
            th_indexes = [i for i in range(len(current_values )) if current_values[i] > new_th] #returns the indexes of the values that are greater than new_th
            tmp_values = list(map(current_values.__getitem__,th_indexes)) #take all the values greater than new_th
            tmp_ref = list(map(el[f"window {window_dim}"]['ref'].__getitem__,th_indexes)) #take all the references that return a value greater than new_th
            tmp_sentences = list(map(el[f"window {window_dim}"]['sentence'].__getitem__,th_indexes)) #take all the sentence parts that return a value greater than new_th
            #print(words_in_window, tmp_sentences, tmp_values)
            if words_in_window in tmp_sentences: #then we have a match with a topic and the words in the window
                #win_results = el[f"window {window_dim}"] #all the results that we have for a given window dimension
                #values = win_results['value']   #all the values referred to the topics that return result greater than a threshold
                index = tmp_sentences.index(words_in_window) #since we have at least one topic > threshold, we take the first index of the first topic that match
                n_possible_topics = tmp_sentences.count(words_in_window) #all the possible topics that returns a result greater than threshold
                all_indexes = [i for i in range(index, index + n_possible_topics)]
                all_best_values = tmp_values[index : index + n_possible_topics] #all the values of the topics that match with the words in the window
                for value,index in zip(all_best_values,all_indexes): #check if the current topic has a match. If no, skip it and go to the next one by following the priorities
                    ref = tmp_ref[index] #reference associated to the result
                    tmp_topic = [key for key in all_refs.keys() for tmp_ref in all_refs[key] if ref == tmp_ref][0] #extract the topic associated with reference
                    if topic_name != tmp_topic: #check if another topic is greater than new_th
                        continue
                    else:
                        sentence_labels['topics&ref'].append((topic_name,ref)) #save the result
                        sentence_labels['matching_words'].append(words_in_window)
                        sentence_labels['values'].append(value)
                        sentence_labels['priority_levels'].append(topic_index+1)
                        current_word_index = current_word_index + window_dim + 1
                        break_flag = True
                        break #break the loop and restart with another topic
                topic_index = topic_index + 1 #restart from next topic
                if break_flag == True:
                    topic_index = 0 #we found the topic, restart the topic search from the current_word_index and reset the break_flag
                    break_flag = False
            else:
                topic_index = topic_index + 1
                if topic_index >= len(topics_ordered):
                    current_word_index = current_word_index + 1
                    topic_index = 0
                continue #try with another topic. Changing topic means that we change the size of window so we take different words to compare. If no topics match, go to the next word
        final_labels.append(sentence_labels)
    return final_labels


labels = process_results_priority_policy_avg_windows(avg_windows=average_windows,result=results, new_th=.7)
filename_labels_avg = "results_priority_policy_avg_windows.pkl"
print("writing to file...")
pickle.dump(labels  , open(os.path.join(path,filename_labels_avg),'wb'))

for label in labels:
    print(label, "\n \n \n \n")

In [ ]:
a = list(np.random.randint(1000,size=100))
b = [(a,i) for i in range(len(a))]
ordered_res = sorted(a,reverse=True)
final_res = [(ordered_res[i],a.index(ordered_res[i])) for i in range(len(a))]
#print(final_res)
#print([heapq.nlargest(len(a) , a)[i], a.index())
c = []
print(sorted(c), max(c))

In [ ]:
################################### TEST
def adec(fun):
    def wrapper(b):
        res = fun(b)
        res = res + 1
        return res
    return wrapper

@adec
def f(c):
    return c+1
print(f(3))
'''
def a_decorator_passing_arbitrary_arguments(function_to_decorate):
    def a_wrapper_accepting_arbitrary_arguments(*args,**kwargs):
        print('The positional arguments are', args)
        print('The keyword arguments are', kwargs)
        function_to_decorate(*args)
    return a_wrapper_accepting_arbitrary_arguments

@a_decorator_passing_arbitrary_arguments
def function_with_no_argument():
    print("No arguments here.")

function_with_no_argument()

def decorator_maker_with_arguments(decorator_arg1, decorator_arg2, decorator_arg3):
    def decorator(func):
        def wrapper(function_arg1, function_arg2, function_arg3) :
            "This is the wrapper function"
            print("The wrapper can access all the variables\n"
                  "\t- from the decorator maker: {0} {1} {2}\n"
                  "\t- from the function call: {3} {4} {5}\n"
                  "and pass them to the decorated function"
                  .format(decorator_arg1, decorator_arg2,decorator_arg3,
                          function_arg1, function_arg2,function_arg3))
            return func(function_arg1, function_arg2,function_arg3)

        return wrapper

    return decorator

pandas = "Pandas"
@decorator_maker_with_arguments(pandas, "Numpy","Scikit-learn")
def decorated_function_with_arguments(function_arg1, function_arg2,function_arg3):
    print("This is the decorated function and it only knows about its arguments: {0}"
           " {1}" " {2}".format(function_arg1, function_arg2,function_arg3))

decorated_function_with_arguments(pandas, "Science", "Tools")
'''
s = "Hello, how are you?"
ind=1
w_size=1
s_split = s.split()
print(s_split)
new_s = ' '.join(s_split[(ind+1):(ind+1+w_size)])
print(new_s)

In [ ]:
path = r"C:\Users\ariel\Documents\PHD\GitHub\semantic_similarity\notebooks"
filename_best = "all_processed_sentences_best.pkl"

if(os.path.exists(os.path.join(path,filename_best))):    #load data already processed
    results = pickle.load(open(os.path.join(path,filename_best),'rb'))
else:
    print("Invalid path")

def adjust_results(results, path ='', new_filename='', dim_max=10):   #current version works only if results are computed with treshold = 0 (we always have a match for each window) and                                                                      #stride = 1
    count = 0
    for key in all_refs.keys():
        count = count + 1
    for res in results:
        sentence = res['whole sentence,speaker,turn'][0]
        s_split = sentence.split()
        #print(s_split, count)
        d_max = len(s_split) if len(s_split) <= dim_max else dim_max
        start_word_indexes = []
        dim = 1
        while dim <= d_max:
            idx = 0 #dim - 1
            while idx < len(s_split) - dim + 1:
                #tmp_res = res[f"window {dim}"]
                #print(tmp_res)
                for counts in range(count):
                    start_word_indexes.append(idx)
                idx = idx + 1
            res[f"window {dim}"]['start_word_indexes'] = start_word_indexes
            start_word_indexes = []
            #print(len(res[f"window {dim}"]['value']),len(res[f"window {dim}"]['start_word_indexes']), dim)
            assert(len(res[f"window {dim}"]['value']) == len(res[f"window {dim}"]['start_word_indexes']))
            dim = dim + 1
            #last_word_indexes = []
            #print(len(s_split) - dim + 1, len(res[f"window {dim}"]['value']),len(res[f"window {dim}"]['start_word_indexes']))
    #print("writing to file...")
    #pickle.dump(results , open(os.path.join(path,new_filename),'wb'))
    print(results)

adjust_results([results[0]], path = path, new_filename = "new_all_processed_sentences_best.pkl")

In [ ]:
b = [0,0,0,0,1,1,1,1,2,2,2,2,3,3,3,3,4,4,4,4]
res = [i for i in range(len(b)) if b[i]>2][0]
print(res-1)
c = [0,1,2,3]
c[1:1+3]
topics = {'t':[(1,3),(4,5),(2,6)]}
if (1,3) in topics['t']:
    print("Hello")

In [39]:
path = r"C:\Users\ariel\Documents\PHD\GitHub\semantic_similarity\notebooks"
filename_best = "new_all_processed_sentences_best.pkl"

if(os.path.exists(os.path.join(path,filename_best))):    #load data already processed
    results = pickle.load(open(os.path.join(path,filename_best),'rb'))
else:
    print("Invalid path")

# this function process the results obtained previously that contain all the possible matches between the words inside each sentence and all the references (grouped by topic, or gesture)
# by using the sentence similarity. The function must have the following characteristics:
# 1) only the results greater than a certain threhsold new_th are accepted
# 2) in case we have a match between words inside a window and more than one references related to one topic, we take only the best one related to that topi (already true in
# all_processed_sentences_best.pkl file)
# 3) in case we have more topics that have a match with words in a window, we take the topic related to the best match
# 4) the functions has to start with a window_dim equal to 1, then compares the best match of a topic with window one, with the matches found by incrementing the window size: it takes
# the best one
# 5) once we found the best window that matches to best topic, we try to see if, by incrementing n_times the window size, the result for that topic changes more than a given threshold.
# If it changes more than a given threhsold at least one time, that best topic is no more considered valid
# 6) if the topic is no more considered valid, it restarts from window_dim equal to 1 but this time it starts to search considering only from second best topic to below.
# 7) repeats steps 5 and 6 until it find a match that works.
# 8) if it find a match that works, we start a new search by moving the window by 1 and comparing all the results obtained with the current best. If a better one is found, substitute the
# the best window with the one found, otherwise continue to move until the window no more contain the words of the best window.
# 9) finally, if for a word we don't have a match, the algorithm moves to the next word
# TODO add a full stop control

def process_results_best_policy(results, all_refs, new_th=0 ,window_max_dim=10): #the maximum number of words that the window can contain

    #takes all elements of the sentence that have a value greater than threshold
    def take_elements_th(el, window_dim, new_th):
        #print(el)
        all_values = el[f"window {window_dim}"]['value']  #all the values (results of the matches between topics and parts of the sentence
        th_indexes = [i for i in range(len(all_values)) if all_values[i] > new_th] #returns the indexes of the values that are greater than new_th
        values = list(map(all_values.__getitem__,th_indexes)) #take all the values greater than new_th
        refs = list(map(el[f"window {window_dim}"]['ref'].__getitem__,th_indexes)) #take all the references that return a value greater than new_th
        matching_words = list(map(el[f"window {window_dim}"]['sentence'].__getitem__,th_indexes)) #take all the sentence parts that return a value greater than new_th
        start_word_indexes = list(map(el[f"window {window_dim}"]['start_word_indexes'].__getitem__,th_indexes))

        return values, refs, matching_words, start_word_indexes

    #n_tries indicates how many times we want to check that the threshold in maintained trough time
    # t_res contains the result referred to the best topic found in a part of the sentence
    # s_res contains all the results obtained by applying the sentence model over a sentence (only results greater tan a certain threshold are taken)
    # sentence contains the sentence that we are processing
    # max_dim is the maximum size of the window that we can apply over that sentence
    def is_coherent(n_tries, th, s_res, t_res, sentence, max_dim):
        s_splits = sentence.split()
        current_topic = t_res['topic'] #the topic found
        current_topic_value = t_res['best_match'] #the value referred to the topic found
        end_index = t_res['new_last_word_idx'] + 1 #the index of the last word of the topic found
        matching_words = t_res['matching_words'] # a sentence can have more matching_words inside, we have to pick the right ones
        start_index = t_res['old_last_word_idx'] + 1
        if t_res['new_last_word_idx'] == len(s_splits) - 1: #in this case we reached the end and we have nothing else to check (we assume that is coherent)
            return True
        #print("IS COHERENT:",matching_words, s_splits[start_index:end_index])
        #print("Sentence:",sentence)
        #print("t_res", t_res['topic'], t_res['best_match'], t_res['matching_words'],"start", start_index,"end", end_index,"len", len(matching_words),"ind", t_res['new_last_word_idx'])
        #print("COHERENCE",matching_words, s_splits[start_index:end_index])
        assert(matching_words == ' '.join(s_splits[start_index:end_index])) #check if the words we are picking are the same of the matching
        topic_ok = False

        for t in range(1,n_tries+1):
            if end_index + t > len(s_splits) - 1 or len(matching_words.split()) + t > max_dim: #check if, by increasing the window size, we reach the end of the sentence or if we can
                return True                                                                    #try to search results in a greater window
            else:
                window_dim = len(matching_words.split()) + t
                s_results = s_res[f"window {window_dim}"]
                processed_words = ' '.join(s_splits[start_index:(start_index + window_dim)])
                #print("IS COHERENT 2:",t_res['old_match_idx'])
                start_search_index = t_res['old_match_idx'][f"window {window_dim}"] #since we are assuming that the match is still not valid, start to search values from old match_indexes
                #print("IS COHERENT 3:",processed_words, s_results['matching_words'][start_search_index:])
                if not processed_words in s_results['matching_words'][start_search_index:]: #then the topic doesn't exists anymore, so fur sure we have a drop in probabilities
                    return False
                else:
                    match_index = s_results['matching_words'][start_search_index:].index(processed_words)
                    n_possible_topics = 0
                    while s_results['matching_words'][start_search_index:][match_index:][n_possible_topics] == processed_words: #count how many topics match with processed words
                        n_possible_topics = n_possible_topics + 1
                        if n_possible_topics >= len(s_results['matching_words'][start_search_index:][match_index:]):
                            break
                    all_match_values = s_results['values'][start_search_index:][match_index : match_index + n_possible_topics]
                    for value in all_match_values:
                        tmp_ref = s_results['refs'][start_search_index:][s_results['values'][start_search_index:].index(value)] #extract the reference associated with the value
                        tmp_topic = [key for key in all_refs.keys() for ref in all_refs[key] if tmp_ref == ref][0] #extract the topic associated with reference
                        if tmp_topic == current_topic:
                            if current_topic_value - value < th: #then the topic does not change so mach by incrementing the window size, check for another n_try
                                topic_ok = True
                                break
                        else: #check the other topic
                            continue
                    if topic_ok:
                        continue
                    else:
                        return False
        return True #if we reached this point than everything is ok. If the function returns False, than we have to avoid the topic

    #this function checks if the topic found is for sure the best one: since we didn't try to search results that starts after a given word, we are not sure for that
    #the paramters are the same as the previous function except for n_tries
    #the function returns a dictionary which tells if the current topic is the best one; if it is the best one that dictionary contain that topic otherwise
    #it will contain the best topic (i.e. the topic that returns greater results). The dictionary contains also the skipped words in case we the best topic changes by sliding the window
    def is_real_best(s_res, t_res, sentence, max_dim):
        s_splits = sentence.split()
        max_th_drop = 0.2 #for coherence check
        current_topic = t_res['topic'] #the topic found
        current_topic_value = t_res['best_match'] #the value referred to the topic found
        end_index = t_res['new_last_word_idx'] + 1  #the index of the last word of the topic found
        matching_words = t_res['matching_words'] # a sentence can have more matching_words inside, we have to pick the right ones
        start_index = t_res['old_last_word_idx'] + 1
        skipped_words = ''
        end = False
        #tmp_new_match_idx = t_res['old_match_idx'].copy()
        if t_res['new_last_word_idx'] == len(s_splits) - 1: #in this case we reached the end and we have nothing else to check (we assume that is the best topic)
            return {"is_best": True, "topic_result": t_res, "skipped_words":skipped_words}
        assert(matching_words == ' '.join(s_splits[start_index:end_index])) #check if the words we are picking are the same of the matching
        tmp_best_res = t_res.copy()
        #processed_words = ' '.join(s_splits[start_index:(start_index + 1 + window_dim)])
        if len(matching_words.split()) == 1: #in this case we have nothing to check since in the next iteration we will search for other words that will not contains the current one
            return {"is_best": True, "topic_result": t_res, "skipped_words":skipped_words}
        else:
            topics_to_avoid = {'topic & dim': []}
            tmp_topics_to_avoid =  topics_to_avoid.copy()
            for t in range(1,len(matching_words.split())-1):
                '''
                w_d = 1 #window dimension ############################### this block is used to start to search results from the right index
                while t_res['old_last_word_idx'] + w_d <= len(s_splits) - 1 and w_d <= max_dim:    #until we reach the end of the sentence
                    s_results = s_res[f"window {w_d}"]
                    if len(s_results['start_word_indexes']) != 0:
                        tmp = [i for i in range(len(s_results['start_word_indexes'])) if s_results['start_word_indexes'][i] > t_res['old_last_word_idx'] + w_d - 1]
                        if len(tmp) > 0: #it means that we have at least one value that will possibly match
                            tmp_idx_start_match_idx[f"window {w_d}"] = tmp[0]
                        else:
                            tmp_idx_start_match_idx[f"window {w_d}"] = len(s_results['matching_words']) #we can avoid to search again
                        w_d = w_d + 1
                '''
                while True:  #try to search if there exists a topic that is coherent (does not change by th if we increase the window) and that is better than the current one
                    next_result = find_best_topic(s_res, max_dim, t_res['old_match_idx'], start_index + t, sentence, topics_to_avoid = topics_to_avoid)
                    print(next_result)
                    if next_result['new_last_word_idx'] != next_result['old_last_word_idx']: ## then we have a new match
                        #then this topic associated to these words is not coherent, check the next word
                        if not is_coherent(n_tries = 1, th = max_th_drop, s_res = s_res, t_res = next_result, sentence = sentence, max_dim = max_dim): #skip this topic and try to search the next
                            topics_to_avoid['topic & dim'].append((next_result['topic'],next_result['best_window']))
                            continue
                        if next_result['best_match'] > current_topic_value:
                            tmp_best_res = next_result.copy()
                            skipped_words = s_splits[start_index : start_index + t]
                            topics_to_avoid['topic & dim'] = []
                        break #in any case we stop the topic search
                    if topics_to_avoid != tmp_topics_to_avoid: #then we have a new topic to avoid and we can search if there are som other topics better than the current one
                        tmp_topics_to_avoid = topics_to_avoid.copy()
                        continue
                    else:
                        break
                    #if next_result['end']: #we reached the end and we didn't find any result
                    #    break

        return {"is_best": False, "topic_result": tmp_best_res, "skipped_words":skipped_words}

    #this function finds the best topic starting from a given word. The best topic is the topic that, for a certain window_dim that contains some of the words of the sentence,
    #returns the best results. If for a given word we can't find any match, the function returns some default values, otherwise it retuns the best topic, the associated
    #reference and value, the words of the sentence that return the best topic, the window_dim that contains those words, the old and new indexes that tell us where to start
    #at the next iteration for both the sentence and the results related to that sentence. If this function is called and we fund that the returned result is for real the best one (for
    #some define policy) then we update all the indexes, otherwise we maintain the old ones and we restart teh search by not considering the topics that do not respect the policy.
    def find_best_topic(s_res, max_dim, idx_last_matches, idx_last_word, #starting from idx_last_word+1, it will find the window that returns the best topic value
                        sentence, topics_to_avoid ={}): #topics_to_avoid indicates which topics we can't search for (e.g. if we found that a topic does not respect the                                                 #policy). and the referred window_dim on which we can't search for that topic
        s_splits = sentence.split()
        best_match = 0 #initialize the results in case we do not find anything
        best_window = 1
        ref = ''
        topic = ''
        matching_words = ''
        tmp_best_value_match = 0

        tmp_idx_last_matches = idx_last_matches.copy()
        #print(idx_last_word)
        #print(idx_last_matches)
        tmp_idx_last_word = idx_last_word
        window_dim = 1
        end = False #this becomes true when we reached the end of the sentence, i.e. if we are in the last word

        if idx_last_word + window_dim == len(s_splits):
            end = True

        #print(s_res)
        #print(max_dim)
        while idx_last_word + window_dim <= len(s_splits) - 1:    #until we reach the end of the sentence
            #for window in range(1,max_dim + 1):
            s_results = s_res[f"window {window_dim}"]
            #print(idx_last_matches)
            start_search_index = idx_last_matches[f"window {window_dim}"] #we take the data inside s_res from this index since the previous ones were already used for search purposes
            processed_words = ' '.join(s_splits[(idx_last_word+1):(idx_last_word + 1 + window_dim)]) #we start with window = 1
            #print(processed_words, "    idx last match", idx_last_matches[f"window {window_dim}"], "    len results",len(s_results['matching_words']))
            #print("window_dim:",window_dim)
            #print("processed words:",processed_words, "  window_dim:", window_dim, "  dim_max:",max_dim, "  start_search_index:",start_search_index)
            #print("RESULTS:", s_results['matching_words'][start_search_index:], "window_dim:", window_dim, processed_words)
            if idx_last_matches[f"window {window_dim}"] > (len(s_results['matching_words']) - 1): #it means that we have no more results for that window_dim, check with another window
                #print("continuing...", "   window_dim",window_dim, "  idx", idx_last_matches[f"window {window_dim}"])
                if max_dim > window_dim:
                    #print("HERE",max_dim,window_dim)
                    window_dim = window_dim + 1
                    continue
                else:
                    break
            #print(processed_words, s_results['matching_words'][start_search_index:])
            if processed_words in s_results['matching_words'][start_search_index:]: #then we have at least one match
                index = s_results['matching_words'][start_search_index:].index(processed_words) # takes the index of the first match between a topic
                                                                                                 # and processed words starting from the last index where we had a match previously
                #print("index",index)
                n_possible_topics = 0
                #print("HERE:", s_results['matching_words'][start_search_index:][index:], processed_words)
                while s_results['matching_words'][start_search_index:][index:][n_possible_topics] == processed_words: #count how many topics match withe processed words
                    n_possible_topics = n_possible_topics + 1
                    #print(n_possible_topics)
                    if n_possible_topics >= len(s_results['matching_words'][start_search_index:][index:]): #avoid taking elements that are outside the array
                        break
                # these values are associated with the topics that match the # processed words
                all_match_values = s_results['values'][start_search_index:][index : index + n_possible_topics]
                ordered_values = sorted(all_match_values, reverse=True) #order the values that match from the highest to the lowest
                #print(ordered_values,tmp_best_value_match)

                #tmp_idx_last_matches[f"window {window_dim}"] = start_search_index + index + len(ordered_values) - 1
                #print("inside fun indx matches:",idx_last_matches)
                #print("topics to avoid",topics_to_avoid,"len ordered values", len(ordered_values))
                #if len(ordered_values) == len(topics_to_avoid): #then we increment the window_size and try with other topics
                #    skip_dims.append(window_dim) #we skip until this dimension next iteration
                #    print("topics before breaking:",topics_to_avoid)
                #    print("breaking... Skipping_dim:",skip_dims)
                #    break
                for i in range(len(ordered_values)):
                    tmp_match = ordered_values[i]
                    #print(tmp_match)
                    #print("match:",tmp_best_match,"  window_dim:",window_dim, "  start_search_index", start_search_index)
                    tmp_ref = s_results['refs'][start_search_index:][s_results['values'][start_search_index:].index(tmp_match)] #reference associated to best result
                    tmp_topic = [key for key in all_refs.keys() for ref in all_refs[key] if tmp_ref == ref][0] #extract the topic associated with reference
                    #print("FINAL CHECK:",(tmp_topic,window_dim), "FINAL CHECK 2:",topics_to_avoid['topic & dim'])
                    if (tmp_topic,window_dim) in topics_to_avoid['topic & dim']: #try with the next ordered value because this one referred to window_dim does not respect he policy
                        #print("SKIPPED TOPIC")
                        continue
                    else:
                        if tmp_match > tmp_best_value_match: #then save the current best
                            #print("OKOKOK")
                            tmp_idx_last_word = idx_last_word + window_dim #Index of the last processed word.
                            tmp_best_value_match = tmp_match
                            matching_words = processed_words
                            best_window = window_dim
                            best_match = tmp_best_value_match
                            ref = tmp_ref
                            topic = tmp_topic
                            break # stop the search for this window_dim, try with a bigger window

            if max_dim > window_dim:
                window_dim = window_dim + 1
            else: #we dindn't find any result even with the maximum window size, so we didn't find any match for a word and we can try with the next one (in the main)
                break

        # save the new indexes from which to start the search for each window. These are temporary since it may happen that the best
        # topic found does not respect the policy (in this case indexes must not be updated).
        # We search for the first match that has a "last_word_index" greater than tmp_idx_last_word ([tmp_idx_last_word + 1][0). Since start index starts from
        # idx_last_word + 1, here we have to compensate by subtracting 1
        window_dim = 1
        while idx_last_word + window_dim <= len(s_splits) - 1 and window_dim <= max_dim:    #until we reach the end of the sentence
            s_results = s_res[f"window {window_dim}"]
            if len(s_results['start_word_indexes']) != 0:
                #print("IDX LAST WORD:",tmp_idx_last_word)
                tmp = [i for i in range(len(s_results['start_word_indexes'])) if s_results['start_word_indexes'][i] > tmp_idx_last_word]#[0] # - 1
                #print(f"TMP INDEXES DIM :{window_dim}, RES:", tmp_idx_last_matches[f"window {window_dim}"])
                if len(tmp) > 0: #it means that we have at least one value that will possibly match
                    tmp_idx_last_matches[f"window {window_dim}"] = tmp[0]
                else:
                    tmp_idx_last_matches[f"window {window_dim}"] = len(s_results['matching_words']) #we can avoid to search again
            window_dim = window_dim + 1

        #print(best_match,tmp_idx_last_word,idx_last_word )
        return {'best_match':best_match, 'best_window':best_window, 'reference':ref, 'topic':topic, 'matching_words':matching_words, 'new_match_idx':tmp_idx_last_matches,
                'old_match_idx':idx_last_matches, 'new_last_word_idx':tmp_idx_last_word, 'old_last_word_idx':idx_last_word, 'end': end, 'topics_to_avoid': topics_to_avoid}

    all_labels = [] #it will contain all the labels for all the sentences
    max_th_drop = 0.2  #if we find that the best topic x does not change by this amount for n_tries times, then this is the best topic. Otherwise we can try by taking the second best topic and by repeating the check
    #all_res = []
    for ind,el in enumerate(results):
        sentence_labels = {'sentence':'', 'topics&ref':[],  'matching_words':[], 'values':[], "skipped_words":[]} #it contains a sentence labelled
        sentence = el['whole sentence,speaker,turn'][0]
        speaker = el['whole sentence,speaker,turn'][1]
        turn = el['whole sentence,speaker,turn'][2]

        sentence_labels['sentence'] = sentence
        sentence_splits = sentence.split()
        final_word_index = len(sentence_splits) - 1 #index of the last word (starting from 0)
        last_processed_word_idx = -1
        processed_words = sentence_splits[0]
        sentence_results = {}
        if len(sentence_splits) < window_max_dim:
            window_max_d = len(sentence_splits)
        else:
            window_max_d = window_max_dim

        last_match_indexes = {}
        #for w in sentence_results.keys():
        #    print(sentence_results[w],"\n \n \n")
        for i in range(1,window_max_d + 1): #take all the data related to the matches greater than a threshold for a given sentence
            values, refs, matching_words, start_word_indexes = take_elements_th(el, i, new_th)
            sentence_results[f"window {i}"] = {'matching_words':[], 'refs':[], 'values':[]}
            sentence_results[f"window {i}"]['values'] = values
            sentence_results[f"window {i}"]['matching_words'] = matching_words
            sentence_results[f"window {i}"]['refs'] = refs
            sentence_results[f"window {i}"]['start_word_indexes'] = start_word_indexes
            last_match_indexes[f"window {i}"] = 0 #this dict will contain, for each window_dim, the index of the first element from which we will start the search in that window
                                                  #every time we use data, we update this indexes to avoid searching data that is no more useful

        topics_to_avoid = {'topic & dim': []}

        counter = 0
        '''
        print(sentence,"\n \n \n")
        for w in sentence_results.keys():
            print(sentence_results[w],"\n \n \n")
        '''
        while True and counter<10:
            #print("counter:",counter)
            #print("topics_before:",topics_to_avoid)
            #print("last match idx before:",last_match_indexes)
            result_dict = find_best_topic(sentence_results, window_max_d, last_match_indexes, last_processed_word_idx, sentence, topics_to_avoid)
            #print("last_idx:",result_dict['old_last_word_idx'],'new_idx:',result_dict['new_last_word_idx'], result_dict['topic'])
            #print(last_processed_word_idx)
            #print(result_dict)
            if result_dict['new_last_word_idx'] != result_dict['old_last_word_idx']: #then we found a match that respects the policy starting from a word
                #print(result_dict, "\n \n \n")
                #print(is_coherent(n_tries = 1, th = max_th_drop, s_res = sentence_results, t_res = result_dict, sentence = sentence, max_dim = window_max_d))
                if is_coherent(n_tries = 1, th = max_th_drop, s_res = sentence_results, t_res = result_dict, sentence = sentence, max_dim = window_max_d):
                    #print("OK:",result_dict,"\n \n \n")
                    #print("LAST WORD IDX",last_processed_word_idx)
                    check = is_real_best(s_res = sentence_results, t_res = result_dict, sentence = sentence, max_dim = window_max_d)
                    if check["is_best"]: #then the current topic is for real the best one
                        sentence_labels['topics&ref'].append((result_dict['topic'],result_dict['reference']))
                        sentence_labels['matching_words'].append(result_dict['matching_words'])
                        sentence_labels['values'].append(result_dict['best_match'])
                        last_processed_word_idx = result_dict['new_last_word_idx']
                        last_match_indexes = result_dict['new_match_idx'].copy()
                    else:
                        new_topic = check["topic_result"]
                        sentence_labels['topics&ref'].append((new_topic['topic'],new_topic['reference']))
                        sentence_labels['matching_words'].append(new_topic['matching_words'])
                        sentence_labels['values'].append(new_topic['best_match'])
                        sentence_labels['skipped_words'].append(check['skipped_words']) #TODO manage the skipped words
                        last_processed_word_idx = new_topic['new_last_word_idx']
                        last_match_indexes = new_topic['new_match_idx'].copy()
                        #print("HERERHRHERHE"
                    topics_to_avoid['topic & dim'] = [] #in botch cases we found the best result
                    #print("LAST WORD IDX",last_processed_word_idx)
                else:
                    #print("PROBLEM:",result_dict,"\n ")
                    #print("last match idx after:",last_match_indexes)
                    topics_to_avoid['topic & dim'].append((result_dict['topic'],result_dict['best_window']))
                    #print("NEW TOPICS TO AVOID:",topics_to_avoid, "\n \n \n")
                    #print("topics_to_append:",result_dict['topic'])
                    #print("topics_before:",topics_to_avoid)

                    #print("topics_after:",topics_to_avoid)

            #in this case we didn't find any match, go to the next word
            else:
                topics_to_avoid['topic & dim'] = []
                last_processed_word_idx = last_processed_word_idx + 1

            if result_dict['end'] == True: #we reached the end
                #print("ending...")
                break
            counter = counter + 1

        all_labels.append(sentence_labels)

    return all_labels


labels = process_results_best_policy([results[1]], all_refs, new_th=0.4)
for el in labels:
    print(el,"\n \n \n")
#print(labels)

 My husband, who is also going to be giving another perspective in today's conversation. Today we're bringing you an amazing, long, English conversation, so prepare your ears, prepare your mind. We're going to be talking about 12 different topics, and hopefully providing some new expressions and new ways to think about life. I don't know about that, but at least some English help for you. 
 
 

{'matching_words': ['My', 'My', 'My', 'My', 'My', 'My', 'My', 'My', 'My', 'My', 'My', 'My', 'My', 'My', 'My', 'My', 'My', 'My', 'My', 'My', 'My', 'My', 'My', 'My', 'My', 'My', 'My', 'My', 'husband,', 'husband,', 'husband,', 'husband,', 'husband,', 'husband,', 'husband,', 'husband,', 'husband,', 'husband,', 'husband,', 'husband,', 'husband,', 'husband,', 'husband,', 'husband,', 'husband,', 'husband,', 'husband,', 'husband,', 'husband,', 'husband,', 'husband,', 'husband,', 'husband,', 'husband,', 'husband,', 'husband,', 'who', 'who', 'who', 'who', 'who', 'who', 'who', 'who', 'who', 'who', 'who', '

In [5]:
##### labelling
path = r"C:\Users\ariel\Documents\PHD\GitHub\semantic_similarity\notebooks"
filename_best = "new_all_processed_sentences_best.pkl"

if(os.path.exists(os.path.join(path,filename_best))):    #load data already processed
    results = pickle.load(open(os.path.join(path,filename_best),'rb'))
else:
    print("Invalid path")
#print(all_refs.keys())

all_refs = {"r_greet":reference_sentence_greet, "r_faster":reference_sentence_faster, "r_goaway":reference_sentence_goaway,
           "r_cigarette":reference_sentence_cigarette, "r_idk":reference_sentence_idk, "r_no":reference_sentence_no,
           "r_yes":reference_sentence_yes, "r_deitic":reference_sentence_deitic,"r_run":reference_sentence_run,
           "r_stop":reference_sentence_stop, "r_adaptor":reference_sentence_adaptor, "r_telephone":reference_sentence_telephone,
           "r_come":reference_sentence_come, "r_drink":reference_sentence_drink, "r_exulting":reference_sentence_exulting,
            "r_link":reference_sentence_link, "r_apologize":reference_sentence_apologize, "r_attention":reference_sentence_attention,
            "r_beg":reference_sentence_beg, "r_careless":reference_sentence_careless, "r_impossible":reference_sentence_impossible,
            "r_walk":reference_sentence_walk,"r_thin":reference_sentence_thin, "r_later":reference_sentence_later,
            "r_lot":reference_sentence_lot, "r_time":reference_sentence_time, "r_clap":reference_sentence_clap,"r_ita":reference_sentence_ita}

topic-gesture_description = {"Greeting":("r_greet", "the gesture that you use when you greet people"), "Talk faster":("r_faster", "the gesture that you use when you want someone to"
                        "talk faster (for example by keeping one or two hands ahead, palms facing you and wrist rotating them recursively)"), "Go away":("r_goaway", "the"
                        "gesture that you use when you want someone to go away from you (for example by keeping one or two hands ahead, palms facing you and by moving the hand back and forth"
                        "starting from the palm facing down)"), "Cigarette":("r_cigarette","the gesture that you do when you want to show smoking intentions (bring two fingers"
                        "close to the mouth)"), "I don't know":("r_idk", "moving the shoulders up and down or move both the arms in front of you with palms facing up, used when you want to"
                        "to show that you don't know something"), "No":("r_no", "moving the head or a finger left and right when you want to say no"), "Yes":("moving the head up and" \
                        "down when you are agreeing for something or you want to say yes"), "Pointing something":("r_deitic","the gesture that you use for indicating something (with head, finger, etc.)"),
                        "Pointing yourself":("r_adaptor","the gesture that you use to point yourself or a part of your body"), "Run": ("r_run","moving arms aback and forth while" \
                        "maintaining an angle of 90° with the elbow to simulate a running movement"), "Walk":("r_walk","moving the index and the middle fingers back and forth to simulate" \
                        "a walking movement"), "Stop":("r_stop","gesture that you make with arms to tell someone that he as to slow down or stop to do something (for example with arms ahead," \
                        "back of the hand facing you and while moving back and forth)"), "Call":("r_telephone", "the gesture that you make to simulate a phone call, for instance by keeping" \
                        "up the thumb and pinky fingers of one hand and by bringing the hand close to an ear"), "Come to me":("r_come", "the gesture that you make to someone to tell him/her" \
                        "to get close, for instance by keeping an arm straight, palm facing down and my moving banck and forth the fingers"), "Drinking":("r_drink", "the gesture that you make" \
                        "to simulate someone drinking, for instance by keeping up the thumb and pinky fingers of one hand and by bringing the hand close to the mouth"), "Exulting":("r_exulting",
                        "the gesture that you make when you are exulting (for example by putting both your arms up and by putting your keeping your hands in the shape of a fist)"), "They are related":
                        "the gesture that you make to tell that 2 people or objects are related (for example by making 2 fists and by bringing them close to each other)"), "Apologize":("r_apologize",
                        "the gesture that you make when you ask someone to forgive you, for instance by tilting the upper body or by keeping arms up, palms facing up and elbows bent so that they" \
                        "form a 90° angle"), "Attention please":("r_attention","the gesture that you make when you ask attention to a person or group of people, for instance by keeping up the" \
                        " index while in front of you"), "Begging you":("the gesture that you make when you are begging someone, for instance by keeping arms in a pray position"), "I don't care":
                        "the gesture that you make when you are not interested in something (for example by bringing the back of one hand under the chin and by moving it back and forth"),
                        "It's impossible": ("r_impossible","the gesture that you make when you don't believe that something is possible, for instance by bending one elbow in a 90° " \
                        "angle, palm of that arm facing up, and by keeping up only the index and the thumb fingers"), "It's thin": ("r_thin","keeping only the pinkie of one hand up while maintaining the other" \
                        "indexes down and the elbow bent to say that something is thin"), "I'll do it later": ("r_later", "keeping a hand in front of you with only the index up to communicate to someone" \
                        "that somthign will happen later and not now"), "It's a lot":("r_lot":"the gesture that you make when you think that something is a lot, for example by keeping a hand in front of you" \
                        "with palm facing you, and by moving it up and down"), "It's late":("r_time", "indicating the wrist with an index to say that it's late for something"), "Congratulations": ("r_clap",
                        "clapping with hands to congratulate with someone"), "What are you saying?":("r_ita", "keep one hand in front of you, palm facing you, the thumb is over the other fingers and the hand" \
                        "is moving back and forth. Used, especially in italy, to tell someone that what is he saying does not make any sense")}

print("In the next there will be shown to you some sentences. Imagine that you are saying the words in the sentence and to gesticulate while saying them. Label the sentences by writing"
      "down the words and the gesture that you imagine to do while saying those words. You can only choose among the following gestures ")
for res in results:
    print("")
    #res['whole sentence,speaker,turn'][0]
nome = input(all_refs.keys())

In [23]:
a = {'f':[1,2],'g':[3,4]}
b = a
b['f'][0]=6
print(a, b)

{'f': [6, 2], 'g': [3, 4]} {'f': [6, 2], 'g': [3, 4]}


In [ ]:
a = {'f':{'m':'cao','v':2},'g':{'m':'cio','v':4},'p':{'m':'ciao','v':1}}
elements = a.items()
print(list(a.keys()),list(elements))
#print([x for x in list(a.keys())])
mmm = map(lambda el: el[1]['v'], list(elements))
print(list(mmm))

#sort_a = sorted(a.items(), key=lambda x: x[1]['v'], reverse=True)
#print(sort_a)
#print(a,"\n",sorted(a,key=a['v']))

In [ ]:
def process_results_best_policy(result, all_refs, new_th=0):
    #print(len(result))
    all_labels = [] #it contains all the labels for all the sentences
    n_tries = 1 #after we find the best matching topic for a set of words, we check if the topic changes by incrementing the window_size: If n_tries is = 2, then the
                #best topic must remain the same for 2 times. For instance, if we have the sentence "Hi, how are you?" and we find that the best topic for "Hi" is "greeting"
                #then also for "Hi, how" and "Hi, how are" the topic must be greeting.
    max_n = 2 #it takes the first max_n best topics. If we try to increment the window size and we find that the topic changes more than a given threshold, get the second best topic and repeat the check
    mex_th_between_topic = 0.2  #if we find that the best topic x does not change by this amount for n_tries times, then this is the best topic. Otherwise we can try by taking the second best topic and by repeating the check


    for el in result:
        sentence_labels = {'sentence':'', 'topics&ref':[], 'words':[], 'values':[]} #it contains a sentence labelled
        sentence = el['whole sentence,speaker,turn'][0]
        speaker = el['whole sentence,speaker,turn'][1]
        turn = el['whole sentence,speaker,turn'][2]
        sentence_labels['sentence'] = sentence

        sentence_splits = sentence.split()
        final_word_index = len(sentence_splits)
        window_dim = 1 #the size of the window (i.e. number of words). We check if there is a match between these words and the reference sentences
        current_words = sentence_splits[0] # We start to analyze the first word, so with window dim = 1
        last_current_word_index = 1 #this contain the index of the last word that have analyzed (it is the last word of the set of words that had a match or the next word of a word that didn't have a match
        last_current_word_index_break = 1 #this is used in case we do not find any instance to avoid breaking the most external while; it contains the index of the of the last
                                          #word that was checked for sentence similarity. We tried all to find a similarity for all the windows but we do not find anything, so
                                          #we continue with the next word

        index_loop = 1

        while last_current_word_index <= final_word_index: # if the condition is false we reached the end and we can analyze the next sentence
            #print(current_words,  el[f"window {window_dim}"]['sentence'], not current_words in el[f"window {window_dim}"]['sentence'])
            if not current_words in el[f"window {window_dim}"]['sentence']: #the words that we are analyzing do not have any match with references -> increment window_size
                #print("HERE", current_words, el[f"window {window_dim}"]['sentence'], last_current_word_index)
                window_dim = window_dim + 1
                if last_current_word_index == final_word_index: #we reached the end of the sentence. Notice that it doesn't mean that we finished the analysis but only to try
                                                                #to find a match with a window that contains all the words from a given word to the end. Now we have to try with
                                                                #the next word, so with window dim=1
                    window_dim = 1 # restart again to check matches from the next word and window_size = 1
                    #print(sentence,last_current_word_index_break, current_words)
                    if last_current_word_index_break == final_word_index: # in this case we can't try anything more, exit from the main loop and analyze another sentence
                        last_current_word_index = last_current_word_index + 1 #this causes the break of the main loop, last_current_word_index is already referred to last word
                    else:
                        current_words = sentence_splits[last_current_word_index_break]  #restart from next next word
                        last_current_word_index_break = last_current_word_index_break + 1 #update the index (now it id referred to the new current word)
                        last_current_word_index = last_current_word_index_break #update the index
                    continue
                last_current_word_index = last_current_word_index + 1 #try by adding a word
                current_words = current_words + ' ' + sentence_splits[last_current_word_index-1]
                continue
            else:
                #print("HERE2", current_words)
                window_dim_tmp = window_dim
                win_results = el[f"window {window_dim}"] #all the results that we have for a given window dimension
                n_possible_topics = win_results['sentence'].count(current_words) #all the possible topics that returns a result greater than threshold
                #print(n_possible_topics )
                values = win_results['value']   #all the values referred to the topics that return result greater than a threshold
                index = win_results['sentence'].index(current_words) #in the case that we have at least one topic > threshold, we take the first index of topics that match
                #print("mmmmmm", values[index : index + n_possible_topics])
                #best = max(values[index : index + n_possible_topics]) #score of the topic that returns the best result
                all_bests = heapq.nlargest(max_n , values[index : index + n_possible_topics]) #the first max_n best topics
                n_max_index = 0
                best = all_bests[n_max_index] #the best topic
                #print(best)
                best_index = values.index(best) #the best topic
                ref = win_results['ref'][best_index] #reference associated to best result
                #print(ref)
                topic = [key for key in all_refs.keys() for tmp_ref in all_refs[key] if ref == tmp_ref][0] #extract the topic associated with reference
                #print(topic)
                #print(final_word_index)
                tmp_last_current_word_index = last_current_word_index #tmp_last_current_word_index contain the index of the last of the words that return the maximum similarity with reference sentences
                #print("TMP LAST CURRENT WORD", tmp_last_current_word_index)
                words = current_words #the words tha that matched with the topic
                while last_current_word_index <= final_word_index :# and index_loop<200: #try to see if, by incrementing the window size, the next best topic returns a greater result.
                    #index_loop = index_loop + 1
                    #print("HERE3")
                    window_dim = window_dim + 1
                    last_current_word_index = last_current_word_index + 1
                    #print(last_current_word_index)
                    if last_current_word_index > final_word_index:  #I will always enter one time in this if. If I enter here, I tried all windows and take the best one
                        #print("I must not be here")
                        if best > new_th: #avoid to append results that are worse than new_th
                            for i in range(n_tries): #check if the topic is the same even if we try bigger window dim
                                if not tmp_last_current_word_index < final_word_index: # then we cannot check anymore so we save the best results
                                    sentence_labels['topics&ref'].append((topic,ref))
                                    sentence_labels['words'].append(words)
                                    sentence_labels['values'].append(best)
                                    break
                                else:
                                    current_tmp_words = words + ' ' + sentence_splits[tmp_last_current_word_index]
                                    if not current_tmp_words in win_results['sentence']: #then we have not any solution. In this case, take another best match and restart the while loop
                                        n_max_index = n_max_index + 1
                                        if n_max_index < n_max and n_max_index < len(all_bests): # in this case we can take the next best value in the list of the first n_best_values (n chosen)
                                            best = all_bests[n_max_index]
                                            window_dim = window_dim_tmp #this is the window dim that we had with
                                            win_results = el[f"window {window_dim}"]
                                            values = win_results['value']
                                            best_index = values.index(best)
                                            ref = win_results['ref'][best_index]
                                            topic = [key for key in all_refs.keys() for tmp_ref in all_refs[key] if ref == tmp_ref][0]
                                            last_current_word_index = last_current_word_index_break #restart from where we break
                                            tmp_last_current_word_index = last_current_word_index
                                            n_max_index = 0
                                            break
                                    #tmp_last_current_word_index = tmp_last_current_word_index + 1
                                    #window_dim = window_dim + 1
                                    #values = win_results['value']
                                    #n_possible_topics = win_results['sentence'].count(current_tmp_words)
                                    #index = win_results['sentence'].index(current_tmp_words)
                                    #best_tmp = max(values[index : index + n_possible_topics])
                                    if best == best_tmp:
                                        continue
                                    else:



                        #print("HERE 4!", best, tmp_last_current_word_index)
                        if tmp_last_current_word_index < final_word_index: #otherwise we cannot continue
                            window_dim = 1      #restart the search from the word next to the previous best window that labels the sentence
                            current_words = sentence_splits[tmp_last_current_word_index]
                            #print(current_words)
                        last_current_word_index = tmp_last_current_word_index + 1
                        last_current_word_index_break = last_current_word_index
                        #print("HEREn",last_current_word_index, current_words)
                        break
                    else:
                        win_results = el[f"window {window_dim}"]
                        #print(win_results)
                        #print("HERE 9")
                        #print("mmmmmmmmm")
                        current_words = current_words + ' ' + sentence_splits[last_current_word_index-1]
                        #print(current_words, win_results['sentence'])
                        if not current_words in win_results['sentence']: #then we have not any solution with current words, try by increasing again the window dim
                            continue
                        values = win_results['value']
                        #print(values)
                        #print(current_words, win_results['sentence'], last_current_word_index)
                        n_possible_topics = win_results['sentence'].count(current_words)
                        #print(n_possible_topics)
                        index = win_results['sentence'].index(current_words)
                        best_tmp = max(values[index : index + n_possible_topics])
                        #print(best_tmp, best)
                        if best_tmp > best: #in this case we have a better result by increasing window dim
                            best = best_tmp
                            window_dim_tmp  = window_dim
                            best_index = values.index(best)
                            ref = win_results['ref'][best_index] #reference associated to best result
                            topic = [key for key in all_refs.keys() for tmp_ref in all_refs[key] if ref == tmp_ref][0]
                            words = current_words
                            tmp_last_current_word_index = last_current_word_index
                            #print("TMP!!",tmp_last_current_word_index)
                        else:
                            continue #try with another window dim
        all_labels.append(sentence_labels)
    return all_labels